In [2]:
import torch
import numpy as np
from torchvision import transforms

options = {
    (0, 0, 0),
    (0, 0, 45),
    (0, 1, 0),
    (0, 1, 45),
    (1, 0, 0),
    (1, 0, 45),
    (1, 1, 0),
    (1, 1, 45)
}

tensor = np.arange(18).reshape(2, 3, 3)
answer = []

for idx, option in enumerate(options):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=option[0]),
        transforms.RandomVerticalFlip(p=option[1]),
        transforms.RandomRotation(degrees=option[2])
    ])
    answer[idx] = transform(tensor)

TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [7]:
array = np.arange(18).reshape(1, 2, 3, 3)
print(array)

print(np.flip(array, axis=2))
print(np.flip(array, axis=3))

[[[[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]]

  [[ 9 10 11]
   [12 13 14]
   [15 16 17]]]]
[[[[ 6  7  8]
   [ 3  4  5]
   [ 0  1  2]]

  [[15 16 17]
   [12 13 14]
   [ 9 10 11]]]]
[[[[ 2  1  0]
   [ 5  4  3]
   [ 8  7  6]]

  [[11 10  9]
   [14 13 12]
   [17 16 15]]]]


In [12]:
array = np.arange(18).reshape(1, 2, 3, 3)
array2 = np.rot90(array, 1, (2, 3))

aa = np.concatenate((array, array2))

In [13]:
aa.shape

(2, 2, 3, 3)

In [14]:
lst = [[1, 2], [3, 4]]
np.flip(lst, axis=1)

array([[2, 1],
       [4, 3]])

In [1]:
import numpy as np
import torch

In [3]:
array = torch.arange(18).view(1, 2, 3, 3)
print(array)

print(torch.flip(array, dims=[2]))
print(torch.flip(array, dims=[3]))
print(torch.flip(array, dims=[2, 3]))

tensor([[[[ 0,  1,  2],
          [ 3,  4,  5],
          [ 6,  7,  8]],

         [[ 9, 10, 11],
          [12, 13, 14],
          [15, 16, 17]]]])
tensor([[[[ 6,  7,  8],
          [ 3,  4,  5],
          [ 0,  1,  2]],

         [[15, 16, 17],
          [12, 13, 14],
          [ 9, 10, 11]]]])
tensor([[[[ 2,  1,  0],
          [ 5,  4,  3],
          [ 8,  7,  6]],

         [[11, 10,  9],
          [14, 13, 12],
          [17, 16, 15]]]])
tensor([[[[ 8,  7,  6],
          [ 5,  4,  3],
          [ 2,  1,  0]],

         [[17, 16, 15],
          [14, 13, 12],
          [11, 10,  9]]]])


In [16]:
import random
import torch

# array = torch.arange(2217984).view(128, 48, 19, 19)
array = torch.arange(18).view(1, 2, 3, 3)
def trans_board(state):
    if random.randint(0, 7) == 0:
        return state
    elif random.randint(0, 7) == 1:
        return torch.rot90(state, k=1, dims=[2, 3])
    elif random.randint(0, 7) == 2:
        return torch.flip(state, dims=[2])
    elif random.randint(0, 7) == 3:
        return torch.rot90(torch.flip(state, dims=[2]), k=1, dims=[2, 3])
    elif random.randint(0, 7) == 4:
        return torch.flip(state, dims=[3])
    elif random.randint(0, 7) == 5:
        return torch.rot90(torch.flip(state, dims=[3]), k=1, dims=[2, 3])
    elif random.randint(0, 7) == 6:
        return torch.flip(torch.flip(state, dims=[3]), dims=[2])
    elif random.randint(0, 7) == 7:
        return torch.rot90(torch.flip(torch.flip(state, dims=[3]), dims=[2]), k=1, dims=[2, 3])

array = trans_board(array)
print(array)

tensor([[[[     18,      37,      56,  ...,     322,     341,     360],
          [     17,      36,      55,  ...,     321,     340,     359],
          [     16,      35,      54,  ...,     320,     339,     358],
          ...,
          [      2,      21,      40,  ...,     306,     325,     344],
          [      1,      20,      39,  ...,     305,     324,     343],
          [      0,      19,      38,  ...,     304,     323,     342]],

         [[    379,     398,     417,  ...,     683,     702,     721],
          [    378,     397,     416,  ...,     682,     701,     720],
          [    377,     396,     415,  ...,     681,     700,     719],
          ...,
          [    363,     382,     401,  ...,     667,     686,     705],
          [    362,     381,     400,  ...,     666,     685,     704],
          [    361,     380,     399,  ...,     665,     684,     703]],

         [[    740,     759,     778,  ...,    1044,    1063,    1082],
          [    739,     758,  

In [53]:
array = torch.arange(18).view(1, 2, 3, 3)
def test(state):
    return torch.rot90(torch.flip(torch.flip(state, dims=[3]), dims=[2]), k=1, dims=[2, 3])

array = test(array)
print(array)

tensor([[[[ 6,  3,  0],
          [ 7,  4,  1],
          [ 8,  5,  2]],

         [[15, 12,  9],
          [16, 13, 10],
          [17, 14, 11]]]])


In [75]:
import torch
import torch.nn as nn
import numpy as np

loss = nn.CrossEntropyLoss(weight=torch.tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=torch.float32)) # loss(input,target) 순서로 입력!

# target is of size nBatch = 3
# Y (=target) 은 class label 형태로 입력되어야 한다 (ex, 0,1,2 ...)
Y = torch.tensor([16, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# Y_pred are logits, raw data 형태로 입력되어야 한다. 
Y_pred_good = torch.tensor(
    [[0.1, 0.2, 0.1, 0.2, 0.1, 0.2, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2 , 0.3, 0.1, 0.2, 0.2, -100, 0.1, 0.1], # predict class 2
    [0.1, 0.2, 0.1, 0.2, 0.1, 0.2, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2 , 0.3, 0.1, 0.2, 0.2, 0.1, 0.1, 0.1],  # predict class 0
    [0.1, 0.2, 0.1, 0.2, 0.1, 0.2, 0.2, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2 , 0.3, 0.1, 0.2, 0.2, 0.1, 0.1, 0.1],]) # predict class 1

Y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
    [0.1, 0.3, 1.5],
    [1.2, 0.2, 0.5]])

l1 = loss(Y_pred_good, Y)
# l2 = loss(Y_pred_bad, Y)
print(f'Batch Loss1:  {l1.item():.4f}')
print(f'Batch Loss2: {l2.item():.4f}')

# get predictions
# torch.max => input tensor의 elements중 최대 값을 반환한다.
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y},\n Y_pred1: {predictions1}, Y_pred2: {predictions2}')

Batch Loss1:  2.9697
Batch Loss2: 1.6418
Actual class: tensor([16,  0,  1]),
 Y_pred1: tensor([13, 13, 13]), Y_pred2: tensor([0, 2, 0])


In [76]:
import torch
import torch.nn as nn
import numpy as np

loss = nn.CrossEntropyLoss() # loss(input,target) 순서로 입력!

# target is of size nBatch = 3
# Y (=target) 은 class label 형태로 입력되어야 한다 (ex, 0,1,2 ...)
Y = torch.tensor([2, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# Y_pred are logits, raw data 형태로 입력되어야 한다. 
Y_pred_good = torch.tensor(
    [[0.1, 0.2, 3.9], # predict class 2
    [1.2, 0.1, 0.3],  # predict class 0
    [0.3, 2.2, 0.2]]) # predict class 1

Y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
    [0.1, 0.3, 1.5],
    [1.2, 0.2, 0.5]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)
print(f'Batch Loss1:  {l1.item():.4f}')
print(f'Batch Loss2: {l2.item():.4f}')

# get predictions
# torch.max => input tensor의 elements중 최대 값을 반환한다.
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y},\n Y_pred1: {predictions1}, Y_pred2: {predictions2}')

Batch Loss1:  0.2834
Batch Loss2: 1.6418
Actual class: tensor([2, 0, 1]),
 Y_pred1: tensor([2, 0, 1]), Y_pred2: tensor([0, 2, 0])


In [80]:
def CELoss(output, action, advantage):
    size = len(output)
    result = torch.zeros(size)
    for i in range(size):
        value = (-1.0)*torch.log(torch.exp(output[i][action[i].long()]) / torch.sum(torch.exp(output[i])))*advantage[i]
        result[i] = value
    return torch.mean(result)

import torch
import torch.nn as nn
import numpy as np

loss = CELoss # loss(input,target) 순서로 입력!

# target is of size nBatch = 3
# Y (=target) 은 class label 형태로 입력되어야 한다 (ex, 0,1,2 ...)
Y = torch.tensor([2, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# Y_pred are logits, raw data 형태로 입력되어야 한다. 
Y_pred_good = torch.tensor(
    [[0.1, 0.2, 3.9], # predict class 2
    [1.2, 0.1, 0.3],  # predict class 0
    [0.3, 2.2, 0.2]]) # predict class 1

Y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
    [0.1, 0.3, 1.5],
    [1.2, 0.2, 0.5]])

l1 = loss(Y_pred_good, Y, [-1, -1, -1])
l2 = loss(Y_pred_bad, Y, [1, 1, 1])
print(f'Batch Loss1:  {l1.item():.4f}')
print(f'Batch Loss2: {l2.item():.4f}')

# get predictions
# torch.max => input tensor의 elements중 최대 값을 반환한다.
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y},\n Y_pred1: {predictions1}, Y_pred2: {predictions2}')

Batch Loss1:  -0.2834
Batch Loss2: 1.6418
Actual class: tensor([2, 0, 1]),
 Y_pred1: tensor([2, 0, 1]), Y_pred2: tensor([0, 2, 0])


In [88]:
nn.functional.softmax(torch.tensor([5, 3, 0], dtype=torch.float32))

C:\Users\삼성\AppData\Local\Temp\ipykernel_12308\2244725732.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(torch.tensor([5, 3, 0], dtype=torch.float32))


tensor([0.8756, 0.1185, 0.0059])

In [89]:
int('2019_04')

201904

In [7]:
from dlgo.networks.alphago import AlphaGoPolicyResNet
from torchsummary import summary

model = AlphaGoPolicyResNet()

summary(model.cuda(), (48, 19, 19))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 19, 19]         230,400
       BatchNorm2d-2          [-1, 192, 19, 19]             384
              ReLU-3          [-1, 192, 19, 19]               0
            Conv2d-4          [-1, 192, 19, 19]         331,776
       BatchNorm2d-5          [-1, 192, 19, 19]             384
              ReLU-6          [-1, 192, 19, 19]               0
            Conv2d-7          [-1, 192, 19, 19]         331,776
       BatchNorm2d-8          [-1, 192, 19, 19]             384
            Conv2d-9          [-1, 192, 19, 19]         331,776
      BatchNorm2d-10          [-1, 192, 19, 19]             384
             ReLU-11          [-1, 192, 19, 19]               0
           Conv2d-12          [-1, 192, 19, 19]         331,776
      BatchNorm2d-13          [-1, 192, 19, 19]             384
           Conv2d-14          [-1, 192,

In [1]:
import torch

def InverseCELoss(output, action):
    batch_size = len(output)
    feature_size = len(output[0])
    result = torch.zeros(batch_size)
    for i in range(batch_size):
        value = 0
        cum = torch.sum(torch.exp(output[i]))
        for j in range(feature_size):
            value += action[i][j] * torch.log(1 - torch.exp(output[i][j]) / cum)
        result[i] = value
    return (-1.0) * torch.mean(result)

def CELoss(output, action):
    batch_size = len(output)
    feature_size = len(output[0])
    result = torch.zeros(batch_size)
    for i in range(batch_size):
        value = 0
        cum = torch.sum(torch.exp(output[i]))
        for j in range(feature_size):
            value += action[i][j] * torch.log(torch.exp(output[i][j]) / cum)
        result[i] = value
    return (-1.0) * torch.mean(result)



In [25]:
a = torch.tensor([[2, 5, 1], [3, 2, 1], [0.5, 2, 0.1]])
b = torch.tensor([[0.2, 0.5, 0.3], [0.3, 0.2, 0.5], [0.5, 0.2, 0.3]])

In [26]:
CELoss(a, b)

tensor(1.7034)

In [27]:
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()
print(loss_fn(a, b))

tensor(1.7034)


In [28]:
c = torch.tensor([1, 2, 0])
loss_fn = nn.CrossEntropyLoss()
print(loss_fn(a, c))

tensor(1.4301)


In [29]:
f = torch.tensor([[0.0, -1.0, 0.0], [0.0, 0.0, -1.0], [-1.0, 0.0, 0.0]])
loss_fn = nn.CrossEntropyLoss()
print(loss_fn(a, f))

tensor(-1.4301)


In [20]:
d = torch.tensor([[-0.2, -0.5, -0.3], [-0.3, -0.2, -0.5], [-0.5, -0.2, -0.3]])
loss_fn = nn.CrossEntropyLoss()
print(loss_fn(-a, d))

tensor(1.6376)


In [9]:
InverseCELoss(a, b)

tensor(0.7356)

In [23]:
nn.functional.softmax(torch.tensor([2.0, 5.0, 1.0]))

C:\Users\삼성\AppData\Local\Temp\ipykernel_5304\59062038.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(torch.tensor([2.0, 5.0, 1.0]))


tensor([0.0466, 0.9362, 0.0171])

In [24]:
nn.functional.softmax(torch.tensor([-2.0, -5.0, -1.0]))

C:\Users\삼성\AppData\Local\Temp\ipykernel_5304\1765222308.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(torch.tensor([-2.0, -5.0, -1.0]))


tensor([0.2654, 0.0132, 0.7214])

In [19]:
cum1 = torch.sum(torch.exp(a[0]))
cum2 = torch.sum(torch.exp(a[1]))
cum3 = torch.sum(torch.exp(a[2]))
value1 = b[0][0] * torch.log(torch.exp(a[0][0])) / cum1
value2 = b[0][1] * torch.log(torch.exp(a[0][1])) / cum1
value3 = b[0][2] * torch.log(torch.exp(a[0][2])) / cum1
print(value1, value2, value3)

tensor(0.0025) tensor(0.0158) tensor(0.0019)


In [1]:
6//4

1

In [2]:
3//2

1

In [7]:
import numpy as np

a = np.array([[3, 2, 1], [0, 0, 0]])
b = np.array([[1, 2, 3], [1, 1, 1]])

c = np.concatenate([a, b])
c

array([[3, 2, 1],
       [0, 0, 0],
       [1, 2, 3],
       [1, 1, 1]])